In [56]:
import nltk
import json
import numpy as np
import csv
import sqlite3
from nltk.stem.lancaster import LancasterStemmer

stemmer = LancasterStemmer()

In [57]:
#question relatable
related_threshold = 0.5
# probability threshold
ERROR_THRESHOLD = 0.6
# load our calculated synapse values
synapse_file = 'synapses.json' 
with open(synapse_file) as data_file: 
    synapse = json.load(data_file) 
    words = np.asarray(synapse['words'])
    synapse_0 = np.asarray(synapse['synapse0']) 
    synapse_1 = np.asarray(synapse['synapse1'])
    classes = np.asarray(synapse['classes'])
word_token = set(words)


In [58]:
# compute sigmoid nonlinearity
def sigmoid(x):
    output = 1/(1+np.exp(-x))
    return output

# convert output of sigmoid function to its derivative
def sigmoid_output_to_derivative(output):
    return output*(1-output)
 
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

def think(sentence, show_details=False):
    x = bow(sentence.lower(), words, show_details)
    if show_details:
        print ("sentence:", sentence, "\n bow:", x)
    # input layer is our bag of words
    l0 = x
    # matrix multiplication of input and hidden layer
    l1 = sigmoid(np.dot(l0, synapse_0))
    # output layer
    l2 = sigmoid(np.dot(l1, synapse_1))
    return l2

def classify(sentence, show_details=False):
    results = think(sentence, show_details)

    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD ] 
    results.sort(key=lambda x: x[1], reverse=True) 
    return_results =[[classes[r[0]],r[1]] for r in results]
    #print ("%s \n classification: %s" % (sentence, return_results))
    return return_results
def add_question(user_input,class_name):
    f = open("untrained.txt", "a")
    f.write("{},{}".format(class_name,user_input))

In [59]:
# classify("How much money do I have in my bank accounts?")
# classify("How much money can I spare?")
# classify("Can I buy this $2.3M apartment?")
# classify("Can I manage to purchase this $1.6M house?")
# classify("What's the total amount of money in my accounts?")
# classify("Can I buy a $2M ferrari")
# classify("I would love to check my bank balance", show_details=True)
#classify("check budget")

In [60]:
def get_balance(bank_type = 'all'):
    value = 0
    conn = sqlite3.connect('Financial_Mary.db')
    c = conn.cursor()
    
    if bank_type == 'all':
        c.execute('SELECT SUM(balance) FROM bank_accounts')
        value = c.fetchall()[0][0]
        print(value)
    conn.close()

def get_budget():
    conn = sqlite3.connect('Financial_Mary.db')
    c = conn.cursor()
    
    c.execute('SELECT SUM(amount_monthly) FROM in_and_out')
    budget = c.fetchall()[0][0]
    print('Monthly Earning {}'.format(budget))
    print('total')
    conn.close()
    
def is_affordable():
    pass

In [61]:
def ask_clerkie():
    print('=========================================================')
    user_input = input("what's your financial question?\n(type 'quit' to exit.) \n\n")
    if user_input == 'quit':
        print('\nThanks for using clerkie')
        return False
    user_token = clean_up_sentence(user_input)
    related_token = 0
    for token in user_token:
        if token in word_token:
            related_token += 1
    if related_token/len(user_token) >= related_threshold and len(user_token) >= 2:
        question_class = classify(user_input)
        if len(question_class) == 0:
            print("\n sorry, I'm still learning, care to classify the question for me?")
            print("\n0: check balance\n1:check budget:\n2:check affordable")
            class_name = input("0 or 1 or 2? (press enter to skip)\n")
            if class_name == '1' or class_name == '2' or class_name == '0':
                add_question(user_input,class_name)
                print("\nThanks! I've saved your question for future traning")
        elif question_class[0][0] == 0:
            get_balance()
        elif question_class[0][0] == 1:
            get_budget()
        elif question_class[0][0] == 2:
            is_affordable()
        
    else:
        print("\nPlease ask financial related question (more than 2 words)")
    return True
    
    

In [62]:
flag = True
while flag:
    flag = ask_clerkie()
    

what's your financial question?
(type 'quit' to exit.) 

check balance
204000
what's your financial question?
(type 'quit' to exit.) 

check budget
Monthly Earning 500
total
what's your financial question?
(type 'quit' to exit.) 

quit

Thanks for using clerkie


In [63]:
classify('check budget')

[[1, 0.997477601859776]]